In [1]:
from skimage.io import imread
import sys
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import copy
import numpy as np
import time


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.applications import resnet50
#pip install --extra-index-url https://developer.download.nvidia.com/compute/redist nvidia-dali-cuda100
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist nvidia-dali-cuda110
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist nvidia-dali-tf-plugin-cuda110
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.types as types
import nvidia.dali.fn as fn
import os.path
from nvidia.dali.plugin.pytorch import DALIClassificationIterator, LastBatchPolicy#（这个是用于填充作用）
from nvidia.dali.plugin.pytorch import DALIGenericIterator

ModuleNotFoundError: No module named 'tensorflow'

In [1]:
import nvidia.dali.plugin.tf as dali_tf  #DALIDataset
mnist_pipeline = MnistPipeline(BATCH_SIZE, device='gpu', device_id=0)

# Define the model and place it on the GPU
with tf.device('/gpu:0'):

# Define shapes and types of the outputs
     shapes = (
    (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE),
    (BATCH_SIZE))
    dtypes = (
    tf.float32,
    tf.int32)

# Create dataset
   mnist_set = dali_tf.DALIDataset(
    pipeline=mnist_pipeline,
    batch_size=BATCH_SIZE,
    output_shapes=shapes,
    output_dtypes=dtypes,
    device_id=0)


#直接使用fit方法即可

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 12)

# COCO

In [2]:
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.fn as fn
import nvidia.dali.types as types
import numpy as np
import os.path


ModuleNotFoundError: No module named 'nvidia'

# TFrecord

In [5]:
from subprocess import call
import os.path

test_data_root = os.environ['DALI_EXTRA_PATH']
#文件路径
tfrecord = os.path.join(test_data_root, 'db', 'tfrecord', 'train')
batch_size = 16
tfrecord_idx = "idx_files/train.idx"
tfrecord2idx_script = "tfrecord2idx"

if not os.path.exists("idx_files"):
    os.mkdir("idx_files")

if not os.path.isfile(tfrecord_idx):
    call([tfrecord2idx_script, tfrecord, tfrecord_idx])

KeyError: 'DALI_EXTRA_PATH'

In [ ]:
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.fn as fn
import nvidia.dali.types as types
import nvidia.dali.tfrecord as tfrec
import numpy as np

pipe = Pipeline(batch_size=batch_size, num_threads=4, device_id=0)
with pipe:
    #实际上，要写的就是这个生成器，所有计算全在这里完成
    inputs = fn.readers.tfrecord(
        path=tfrecord,
        index_path=tfrecord_idx,
        features={
            "image/encoded" : tfrec.FixedLenFeature((), tfrec.string, ""),
            "image/class/label": tfrec.FixedLenFeature([1], tfrec.int64,  -1),
            "image/class/text": tfrec.FixedLenFeature([ ], tfrec.string, ""),
            "image/object/bbox/xmin": tfrec.VarLenFeature(tfrec.float32, 0.0),
            "image/object/bbox/ymin": tfrec.VarLenFeature(tfrec.float32, 0.0),
            "image/object/bbox/xmax": tfrec.VarLenFeature(tfrec.float32, 0.0),
            "image/object/bbox/ymax": tfrec.VarLenFeature(tfrec.float32, 0.0)})
    
    #
    jpegs = inputs["image/encoded"]
    images = fn.decoders.image(jpegs, device="mixed", output_type=types.RGB)
    resized = fn.resize(images, device="gpu", resize_shorter=256.)
    output = fn.crop_mirror_normalize(
        resized,
        dtype=types.FLOAT,
        crop=(224, 224),
        mean=[0., 0., 0.],
        std=[1., 1., 1.])
    pipe.set_outputs(output, inputs["image/class/text"])

In [6]:
pipe.build()
pipe_out = pipe.run()

NameError: name 'pipe' is not defined

In [ ]:
#可视化
images, labels = pipe_out
show_images(images.as_cpu(), labels)